In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter #但tensorboard好像还是tensorflow的产品

transform = transforms.Compose([transforms.ToTensor()])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./cifar10/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./cifar10/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 0、启动：tensorboard --logdir=runs （如果把runs文件夹保存着，那么随时可以展示结果。）
## 1、展示一批图片，在IMAGES一栏

In [82]:
#默认的log_dir就是runs，存放在本地
writer = SummaryWriter('runs/fuck_tensorboard')

dataiter = iter(trainloader)
images, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(images)
writer.add_image('4_ccifar10_images', img_grid)

## 2、展示网络结构图，输入输出，在GRAPHS一栏

In [19]:
writer.add_graph(net, images)
writer.close() #似乎是要close后才能看到graph

## 3、可视化分析数据库分布，在PROJECTOR一栏

In [92]:
# 可视化分析数据库分布
#（注意有延迟的，可能要打开退出好几次才能看到）
def select_n_random(data, labels, n=100):
    assert len(data) == len(labels)
    perm = torch.randperm(len(data))
    print(data[perm][:n].shape)
    tmp=data[perm][:n].transpose(0,3,1,2)
    return torch.tensor(tmp), torch.tensor(labels)[perm][:n]

images, labels = select_n_random(trainset.data, trainset.targets)
class_labels = [classes[lab] for lab in labels]
features = images.view(-1, 32*32*3)
writer.add_embedding(features,  #就是把特征铺平，然后可视化PCA或者t-SNE
                    metadata=class_labels,
                    label_img=images)
writer.close()

(100, 32, 32, 3)


## 4、画loss之类的，在SCALARS一栏

In [96]:
# 不重要，没必要看。helper functions
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

def images_to_probs(net, images):
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    #就是随机取一批看看预测错的和对的。
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [97]:
running_loss = 0.0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:    
            # 1000组样本记录一次loss
            writer.add_scalar('training loss',
                            running_loss / 1000,
                            epoch * len(trainloader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(net, inputs, labels),
                            global_step=epoch * len(trainloader) + i) #就是好多组figure编个号。
            running_loss = 0.0
print('Finished Training')

Finished Training


## 5、绘制PR曲线，显示在PR CURVES一栏

In [100]:
class_probs = []
class_label = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        output = net(images)
        class_probs_batch = [F.softmax(el, dim=0) for el in output]

        class_probs.append(class_probs_batch)
        class_label.append(labels)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_label = torch.cat(class_label)

# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_label, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_label)

## 6、权重直方图&权值可视化

In [179]:
writer.add_histogram("conv1.weight",net.conv1.weight,0)
writer.close()

kernel=(net.conv1.weight)
print(kernel.shape)
kernel=kernel.view(18,5,5).unsqueeze(dim=1)
print(kernel.shape)
img_grid = torchvision.utils.make_grid(kernel,normalize=True)
writer.add_image('kernel', img_grid)

torch.Size([6, 3, 5, 5])
torch.Size([18, 1, 5, 5])


## 7、特征可视化

In [192]:
img = next(iter(trainloader))[0][0]
writer.add_image("1_img",img)
img=img.unsqueeze(dim=0)
for name, layer in net._modules.items():
    if name=="fc1": #我们只看前两层卷积后的特征
        break
    print(name)
    img=layer(img)
    if "conv" in name:
        print(img.shape)
        tmp=img.transpose(0,1) #把16个通道当成16张图片 #注意torch.transpose是交换两个维度！
        img_grid = torchvision.utils.make_grid(tmp,normalize=True)
        writer.add_image(f'{name}_ffeature_maps', img_grid, global_step=0)

conv1
torch.Size([1, 6, 28, 28])
pool
conv2
torch.Size([1, 16, 10, 10])
feature


## (8、CAM可视化理解，意义不大)

In [201]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.feature = nn.AvgPool2d(5) #
        self.fc1 = nn.Linear(16 * 1 * 1, 10)#


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x0 = self.pool(F.relu(self.conv2(x)))#
        x=self.feature(x0)#
        x = torch.flatten(x, 1) 
        x = self.fc1(x)
        return x,x0#

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

running_loss = 0.0
for epoch in range(10): 

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs,_ = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:    
            writer.add_scalar('fuck loss',
                            running_loss / 1000,
                            epoch * len(trainloader) + i)
            running_loss = 0.0
print('Finished Training')

Finished Training


In [270]:
writer = SummaryWriter('runs/fuck')
img = next(iter(testloader))[0][2].unsqueeze(dim=0) 
writer.add_image("CAM00",img[0])
idx,fea=net(img)
_,idx=torch.max(idx,1)

fea=fea[0] #16，5，5
wei=net.fc1.weight
print(wei.shape)
wei=wei[idx][0]
for i in range(9):
    fea[i]*=wei[i]

torch.Size([10, 16])


In [272]:
import cv2
heatmap = fea.detach().numpy()
heatmap = np.mean(heatmap, axis=0)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
heatmap = cv2.resize(heatmap, (32,32))
real_heatmap = np.uint8(255 * heatmap)
real_heatmap = cv2.applyColorMap(real_heatmap, cv2.COLORMAP_JET)
real_heatmap = real_heatmap.transpose(2,0,1)
real_heatmap = torch.tensor(real_heatmap)
heatmap = torch.tensor(heatmap)
heatmap=heatmap.unsqueeze(dim=0)
res=img[0]*0.5+real_heatmap
writer.add_image("CAM01",heatmap)
writer.add_image("CAM02",real_heatmap)
writer.add_image("CAM03",res)